### Path setup and imports

In [1]:
import matlab.engine
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.io as sio
import os
import process.histogram as hist
import process.resample as res

In [2]:
# Set relevant paths
datapath = os.path.join('/Users', 'danielsprague', 'FOCO_lab', 'data')


### Resample data 

In [5]:
from scipy.interpolate import RegularGridInterpolator
import tifffile
from scipy.ndimage import zoom

'''
Currently takes ~1 min to run with zoom but seems to have good output.
Much faster to run using Matlab GriddedInterpolant, but similar Python
function not running efficiently. Zoom uses order 3 spline interpolation.
TODO: make this more efficient
'''

reffile = sio.loadmat(datapath+ '/NP_FOCO_cropped/2022-02-12-w01-NP1/neuropal_1_MMStack_Pos0.ome.mat')

data = reffile['data']
oldres = np.asarray(reffile['info']['scale'][0][0]).flatten()
channels = [0,2,4,1]
RGBW = np.squeeze(data[:,:,:,channels])
RGBW = RGBW.astype('int32')

newres = [0.35, 0.35, 1.2]

new_im = res.zoom_interpolate(newres, oldres, RGBW)

new_im = new_im.astype('uint16')
new_im = np.transpose(new_im, (2,3,0,1))

tifffile.imwrite(datapath+ '/python_resample_test.tif', new_im, imagej = True)




(28, 4, 221, 919)


### Match histogram of data

In [ ]:
'''
Example code to process through all datasets in NP_FOCO_cropped and match histogram to a reference dataset.
In this case, that reference dataset is 11_YAaLR.mat from the original NeuroPAL datasets
'''


hist_df = pd.DataFrame()

reffile = sio.loadmat(datapath+'/NP_paper/all/11_YAaLR.mat')
refchannels = reffile['prefs']['RGBW'][0][0]-1
refdata = reffile['data']
refRGBW = np.squeeze(refdata[:,:,:,refchannels])

for folder in os.listdir(datapath+'/NP_foco_cropped'):
    if not folder[0:3] == '.DS':
        for file in os.listdir(datapath+'/NP_foco_cropped/'+folder):
            if file[-4:] == '.mat' and file[-6:] != 'ID.mat':
                imfile = sio.loadmat(datapath+'/NP_foco_cropped/'+folder +'/'+file)

                data = imfile['data']
                channels = [0,2,4,1]
                RGBW = np.squeeze(data[:,:,:, channels])
                RGBW = RGBW.astype('int32')

                RGBW_new = hist.match_histogram(RGBW, refRGBW, 4096, 65536)

                sio.savemat(datapath+'/NP_FOCO_hist_match/'+folder+'/'+'hist_match_image.mat', {'Hist_RGBW':RGBW_new})
